<a href="https://colab.research.google.com/github/HNR1/Bachelors-project/blob/main/fid_score_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/HNR1/pytorch-fid.git
!pip install transformers diffusers tomesd accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/HNR1/pytorch-fid.git to /tmp/pip-req-build-dmm9jfvr
  Running command git clone --filter=blob:none --quiet https://github.com/HNR1/pytorch-fid.git /tmp/pip-req-build-dmm9jfvr
  Resolved https://github.com/HNR1/pytorch-fid.git to commit 3c9ea42c486637d51a86a6e75028b47893977520
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from diffusers import DiffusionPipeline
import torch, tomesd, os.path
from pytorch_fid.fid_score import calc_fid_given_lists
from time import time

In [3]:
if os.path.exists("/content/Bachelors-project"):
  %cd Bachelors-project
  !git pull
  %cd ..
else:
  !git clone https://github.com/HNR1/Bachelors-project.git

/content/Bachelors-project
Already up to date.
/content


In [4]:
# parse prompts from prompts.txt
filename1 = "Bachelors-project/prompts.txt"
filename2 = "Bachelors-project/seeds_768x768.txt"
prompts = []
seeds = []
with open(filename1, 'r', encoding='UTF-8') as file:
    while line := file.readline():
        prompts.append(line.rstrip())
with open(filename2, 'r', encoding='UTF-8') as file:
    while line := file.readline():
        seeds.append(int(line.rstrip()))

In [6]:
pipeline = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5").to("cuda")
pipeline.enable_attention_slicing()

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [7]:
prompt = prompts[0]
seed = seeds[0]
image_clean = pipeline(prompt, 768, 768, generator=torch.Generator().manual_seed(seed)).images[0]
tomesd.apply_patch(pipeline, 0.2)
image_20_1 = pipeline(prompt, 768, 768, generator=torch.Generator().manual_seed(seed)).images[0]
image_20_2 = pipeline(prompt, 768, 768, generator=torch.Generator().manual_seed(seed)).images[0]
tomesd.remove_patch(pipeline)
print('Done')

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Done


In [8]:
calc_fid_given_lists(image_clean, image_20_1)

55.81440174534166

In [9]:
calc_fid_given_lists(image_clean, image_20_2)

57.37016430361197

In [10]:
calc_fid_given_lists(image_20_1, image_20_2)

25.513302031801917